<a href="https://colab.research.google.com/github/AHMED-AMINE-GUIIDAT/nlp_tp01/blob/main/NLP_tasks_using_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Natural Language Processing Tasks with Transformers using HuggingFace



**Import packages**

In [2]:
pip install transformers sentencepiece


     |████████████████████████████████| 2.6 MB 30.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 
     |████████████████████████████████| 3.3 MB 57.5 MB/s 
     |████████████████████████████████| 895 kB 78.6 MB/s 
     |████████████████████████████████| 636 kB 63.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


###**DataSet**##


In [3]:
array_texts = [      
            ["I love surfing in hawaii", "English"],
            ["my name is amine I hate my surfing","English"],
            ["في هواي أحب  ركوب الأمواج بشعف" , "Arabic"],
            ["لا أحب  ركوب الأمواج بالمرة", "Arabic"],
            ["J'aime surfer a hawaii ", "French"],
            ["Je n'aime pas surfer", "French"]
]

print(array_texts)

[['I love surfing in hawaii', 'English'], ['my name is amine I hate my surfing', 'English'], ['في هواي أحب  ركوب الأمواج بشعف', 'Arabic'], ['لا أحب  ركوب الأمواج بالمرة', 'Arabic'], ["J'aime surfer a hawaii ", 'French'], ["Je n'aime pas surfer", 'French']]


In [4]:
from transformers import pipeline,AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
model = AutoModel.from_pretrained("akhooli/gpt2-small-arabic")

Some weights of the model checkpoint at akhooli/gpt2-small-arabic were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
my_classifier_using_pipeline = pipeline('sentiment-analysis' ,  model='akhooli/xlm-r-large-arabic-sent')

Some weights of the model checkpoint at akhooli/xlm-r-large-arabic-sent were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Preprocessing data**

In [6]:
import string
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation  #all punctuations 

def preprocessing_data(text , type):
    #remove punctuations
 
  
    if type == "Arabic":

   
      #remove longation and stopwords
      text = re.sub("[إأآا]", "ا", text)
      text = re.sub("ى", "ي", text)
      text = re.sub("ؤ", "ء", text)
      text = re.sub("ئ", "ء", text)
      text = re.sub("ة", "ه", text)
      text = re.sub("گ", "ك", text)

    
    elif type == "English":

      text = text.lower()
      text = ''.join([item for item in text if item not in punctuations ])
      
 
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Text classification (Sentiments Analysis) - English, French and Arabic languages**

In [7]:
for text,language in array_texts:
  res = my_classifier_using_pipeline(preprocessing_data(text , language))[0]
  res['label'] = "Positive" if res['label'] == 'LABEL_2' else ("Negative" if res['label'] == 'LABEL_1'  else "Mixed")
  print("'{}' is {} {} text with occuracy = {}%".format(text,res['label'],language,round(res['score']*100, 2)))

'I love surfing in hawaii' is Positive English text with occuracy = 45.46%
'my name is amine I hate my surfing' is Negative English text with occuracy = 79.3%
'في هواي أحب  ركوب الأمواج بشعف' is Positive Arabic text with occuracy = 43.31%
'لا أحب  ركوب الأمواج بالمرة' is Negative Arabic text with occuracy = 51.13%
'J'aime surfer a hawaii ' is Mixed French text with occuracy = 39.72%
'Je n'aime pas surfer' is Negative French text with occuracy = 83.26%


**Text generation (English, French and Arabic languages)**

In [8]:
from transformers import pipeline, set_seed
generator_english_text = pipeline('text-generation', model='gpt2')
generator_arabic_text = pipeline('text-generation', model='akhooli/gpt2-small-arabic')
generator_french_text = pipeline('text-generation', model='dbddv01/gpt2-french-small')
set_seed(42)

In [9]:
for text , lang in array_texts:
  if lang == "Arabic":
    print(generator_arabic_text(text , max_length=30, num_return_sequences=5))
  elif lang == "English":
    print(generator_english_text(text , max_length=30, num_return_sequences=5))
  else:
    print(generator_french_text(text , max_length=30, num_return_sequences=5))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I love surfing in hawaii! I hope my new family will be there…\n\nAdvertisements'}, {'generated_text': 'I love surfing in hawaii and I\'d love to join an American team someday… but I wish I were there," the 24-year-old'}, {'generated_text': 'I love surfing in hawaii," said Bob Lander, a resident of South Kailua. "I have this beautiful canoe on a lot of'}, {'generated_text': 'I love surfing in hawaii!\n\nAs a way to show what a good place to surf, a small picture of their Facebook group, as'}, {'generated_text': "I love surfing in hawaii, so I'm interested in finding out as much as I can about how far we've come from where we were."}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'my name is amine I hate my surfing\n\nThe sun was high and shining, and the wind whirled, and the wind blew the waves'}, {'generated_text': 'my name is amine I hate my surfing friends, my family, my friends\' families," she writes. "When I got my first surfing challenge'}, {'generated_text': "my name is amine I hate my surfing and would rather surf the sea in my swimsuit\n\nI'm in love. Amine, but"}, {'generated_text': 'my name is amine I hate my surfing. I was going out to check the surf to see if anything I was doing or not was fine.'}, {'generated_text': 'my name is amine I hate my surfing because it kills my sense of smell. But it\'s been the most important of my surfing\'s benefits."'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'في هواي أحب  ركوب الأمواج بشعف و حب لم تكن تعلم، وكان يعرف في البداية باسم آن. كانت الموسيقى الأولى لآمون في عام 1929'}, {'generated_text': 'في هواي أحب  ركوب الأمواج بشعف، حتى من البحر، فصرخ على الشاطئ. وفي الطريق كان هو وعائلته في حالة تأهب إلى'}, {'generated_text': 'في هواي أحب  ركوب الأمواج بشعف جميل شديد، فكان له صديق في منطقة تسولج الواقعة في صحراء النفيس. ومن بعد ذلك شارك'}, {'generated_text': 'في هواي أحب  ركوب الأمواج بشعف. كنت كنت أعمل النظر عن هذا السؤال، أنا لا أحب  ركوب الأمواج وأنا تعلم ان تكون تلك'}, {'generated_text': 'في هواي أحب  ركوب الأمواج بشعف، وكان ابن اخويه يدعى اوفيم و ابن اخويه يسمى ادما . أما هواي فقد'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'لا أحب  ركوب الأمواج بالمرة، في الوقت الذي لم يكن فيه على الأرض شيء سوى أن يجمعها الحظ في البحر. لم تكافئ السفن التي'}, {'generated_text': 'لا أحب  ركوب الأمواج بالمرة، وفي هذه الأثناء كان يرى دائما على طول هذا الطريق من حيث طبيعة التنقلات ، وكانت حياته مليئة بالأحداث، التي'}, {'generated_text': 'لا أحب  ركوب الأمواج بالمرة و كانت دائما تنظر الماء و كان الناس يقولون ان الوفايات و لايعرفونها و يقول الناس ان في'}, {'generated_text': 'لا أحب  ركوب الأمواج بالمرة وهو يصيح في الماء. فقال له: أنت تعلم أن لا إله إلا الله. فقال: إن الله جعلنا'}, {'generated_text': 'لا أحب  ركوب الأمواج بالمرة بعد ان اشارت على ان اروع اساليب التابلوجة. ثم كتب ان كل ما تبقى من تلك الال'}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "J'aime surfer a hawaii ie sha boktai a la terre » (To tori) par les"}, {'generated_text': "J'aime surfer a hawaii utei, et d'où se vit un lieu de repos dans la montagne, où le"}, {'generated_text': "J'aime surfer a hawaii ikini pour le chant comme s'il les accompagne. Cette partie est toujours jouée avec les"}, {'generated_text': "J'aime surfer a hawaii izari ikio shunchi, qui signifie « l'homme qui reçoit les moyens"}, {'generated_text': "J'aime surfer a hawaii ikki. »Une première ligne entre le centre-ville et la ville est inaugurée le 1"}]
[{'generated_text': "Je n'aime pas surfer sur son attitude envers son ex. Le 25 juillet, elle se rend à l'hôtel de ville et participe au"}, {'generated_text': "Je n'aime pas surfer sur le fait que certaines chansons étaient du type Hez, et les morceaux, de façon assez classique, contiennent des"}, {'generated_text': "Je n'aime pas surfer sur les « sources disponibles de » cette écriture ; elle fait dire le choix de son écritur

###**Name entity recognition (NER) English, French and Arabic languages**

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

ner_english_recognition = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")
ner_arabic_recognition = pipeline("ner", model="hatmimoha/arabic-ner", tokenizer="hatmimoha/arabic-ner")
ner_french_recognition = pipeline("ner", model="gilf/french-postag-model", tokenizer="gilf/french-postag-model")

In [11]:
for text , lang in array_texts:
  if lang == "Arabic":
    print(ner_arabic_recognition(text))
  elif lang == "English":
    print(ner_english_recognition(text))
  else:
    print(ner_french_recognition(text))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[]
[{'entity': 'B-PER', 'score': 0.70183057, 'index': 4, 'word': 'am', 'start': 11, 'end': 13}]
[{'entity': 'LABEL_12', 'score': 0.99942654, 'index': 1, 'word': 'في', 'start': 0, 'end': 2}, {'entity': 'LABEL_12', 'score': 0.9935934, 'index': 2, 'word': 'هوا', 'start': 3, 'end': 6}, {'entity': 'LABEL_12', 'score': 0.99405295, 'index': 3, 'word': '##ي', 'start': 6, 'end': 7}, {'entity': 'LABEL_12', 'score': 0.9950589, 'index': 4, 'word': 'أحب', 'start': 8, 'end': 11}, {'entity': 'LABEL_12', 'score': 0.9952591, 'index': 5, 'word': 'ركوب', 'start': 13, 'end': 17}, {'entity': 'LABEL_12', 'score': 0.983526, 'index': 6, 'word': 'الأمواج', 'start': 18, 'end': 25}, {'entity': 'LABEL_12', 'score': 0.99918866, 'index': 7, 'word': 'بش', 'start': 26, 'end': 28}, {'entity': 'LABEL_12', 'score': 0.9994349, 'index': 8, 'word': '##عف', 'start': 28, 'end': 30}]
[{'entity': 'LABEL_12', 'score': 0.9997481, 'index': 1, 'word': 'لا', 'start': 0, 'end': 2}, {'entity': 'LABEL_12', 'score': 0.9996351, 'index':

**Question answering**

In [12]:
from transformers import pipeline

##### English context
question = "What is the capital of the Hawaii?"
context = r"Hawaii, a US state, is an isolated volcanic archipelago in the central Pacific. Its islands are renowned for their rugged landscapes made up of cliffs, waterfalls, tropical forest and beaches with sands of golden, red, black and even green hues. Of the 6 largest islands, O'ahu is home to the largest city, which is also the capital: Honolulu, home to Waikiki Beach, shaped like a crescent moon, and the WWII memorials of Pearl Harbor. "
# Generating an answer to the question in context
qa_english_sentence = pipeline("question-answering")
answer = qa_english_sentence(question=question, context=context)
# Print the answer
print(f"English Question: {question}")
print(f"English Answer: '{answer['answer']}' with accuracy {answer['score']}")

##### French context
qa_french_sentence = pipeline('question-answering', model='fmikaelian/camembert-base-squad', tokenizer='fmikaelian/camembert-base-squad')
question_fr = "dans quel état se situe  Hawaii ?"
context_fr = "Hawaii, un État américain, est un archipel volcanique isolé dans le Pacifique central. Ses îles sont réputées pour leurs paysages accidentés composés de falaises, de chutes d'eau, de forêt tropicale et de plages dont le sable arbore des teintes dorées, rouges, noires, voire vertes. Des 6 plus grandes îles, O'ahu accueille la plus grande ville, qui est également la capitale : Honolulu, où se situent la plage de Waikiki, en forme de croissant de lune, et les mémoriaux de la Seconde Guerre mondiale de Pearl Harbor"
french_answer = qa_french_sentence(question =  question_fr , context = context_fr)
# Print the answer
print(f"French Question: {question_fr}")
print(f"French Answer: '{french_answer['answer']}' with accuracy {french_answer['score']}")


English Question: What is the capital of the Hawaii?
English Answer: 'Honolulu' with accuracy 0.7641329765319824



French Question: dans quel état se situe  Hawaii ?
French Answer: ' américain,' with accuracy 0.21530330181121826


###**Filling masked text in English,French and Arabic languages**

In [13]:
from transformers import pipeline
english_fill_mask = pipeline('fill-mask', model='bert-large-uncased-whole-word-masking')
french_fill_mask  = pipeline("fill-mask", model="camembert-base", tokenizer="camembert-base")
arabic_fill_mask = pipeline('fill-mask', model='CAMeL-Lab/bert-base-camelbert-ca')

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at CAMeL-Lab/bert-base-camelbert-ca were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:

print(english_fill_mask("Hi I'm a [MASK] data scientist."))
print(french_fill_mask("L'examen est <mask>."))
print(arabic_fill_mask("الهدف من الدراسة هو [MASK] ."))

[{'sequence': "hi i'm a computer data scientist.", 'score': 0.4872608184814453, 'token': 3274, 'token_str': 'computer'}, {'sequence': "hi i'm a freelance data scientist.", 'score': 0.04428230598568916, 'token': 15919, 'token_str': 'freelance'}, {'sequence': "hi i'm a research data scientist.", 'score': 0.033498384058475494, 'token': 2470, 'token_str': 'research'}, {'sequence': "hi i'm a data data scientist.", 'score': 0.03199975937604904, 'token': 2951, 'token_str': 'data'}, {'sequence': "hi i'm a math data scientist.", 'score': 0.026594799011945724, 'token': 8785, 'token_str': 'math'}]
[{'sequence': "L'examen est terminé.", 'score': 0.2890785038471222, 'token': 3818, 'token_str': 'terminé'}, {'sequence': "L'examen est ouvert.", 'score': 0.06130659952759743, 'token': 1422, 'token_str': 'ouvert'}, {'sequence': "L'examen est gratuit.", 'score': 0.043661560863256454, 'token': 434, 'token_str': 'gratuit'}, {'sequence': "L'examen est annulé.", 'score': 0.03470303490757942, 'token': 16737, '

**Summarization**

In [16]:
from transformers import pipeline
summarizer = pipeline("summarization")

In [18]:
article_summary = """Hawaii comprises nearly the entire Hawaiian archipelago, 137 volcanic islands spanning 1,500 miles (2,400 km) that are physiographically and ethnologically part of the Polynesian subregion of Oceania.[13] The state's ocean coastline is consequently the fourth longest in the U.S., at about 750 miles (1,210 km).[b] The eight main islands, from northwest to southeast, are Niʻihau, Kauaʻi, Oʻahu, Molokaʻi, Lānaʻi, Kahoʻolawe, Maui, and Hawaiʻi, after which the state is named; it is often called the "Big Island" or "Hawaii Island" to avoid confusion with the state or archipelago. The uninhabited Northwestern Hawaiian Islands make up most of the Papahānaumokuākea Marine National Monument, the nation's largest protected area and the third largest in the world."""

context_fr = "Hawaii, un État américain, est un archipel volcanique isolé dans le Pacifique central. Ses îles sont réputées pour leurs paysages accidentés composés de falaises, de chutes d'eau, de forêt tropicale et de plages dont le sable arbore des teintes dorées, rouges, noires, voire vertes. Des 6 plus grandes îles, O'ahu accueille la plus grande ville, qui est également la capitale : Honolulu, où se situent la plage de Waikiki, en forme de croissant de lune, et les mémoriaux de la Seconde Guerre mondiale de Pearl Harbor."

print("Summary:",summarizer(article_summary, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])
print("Summary:",summarizer(context_fr, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

Summary:  Hawaii comprises nearly the entire Hawaiian archipelago, 137 volcanic islands spanning 1,500 miles (2,400 km) The state's ocean coastline is consequently the fourth longest in the U.S., at about 750 miles (1,210 km) Uninhabited Northwestern Hawaiian Islands make up most of the Papahānaumokuākea Marine National Monument, the nation's largest protected area .
Summary:  Hawaii, un État américain, est un archipel volcanique isolé dans le Pacifique central . Des 6 plus grandes îles, Oahu accueille la plus grande ville : Honolulu, où se situent la plage de Waikiki, en forme de croissant de lune .


**Translation**

In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_ar = "المحكمة العليا للولايات المتحدة"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


['The Supreme Court of the United States.']

**Feature extraction**

In [3]:
from transformers import pipeline, AutoTokenizer

feature_extraction = pipeline('feature-extraction', model="distilroberta-base", tokenizer="distilroberta-base")
features = feature_extraction("i am data scientist")


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
print(features[0])
print(len(features[0])) #6

[[-0.004540705122053623, 0.060030438005924225, -0.011161313392221928, -0.12302959710359573, 0.07138173282146454, -0.13693442940711975, -0.04459403455257416, 0.08033394068479538, 0.05772212892770767, -0.04234227165579796, -0.055484019219875336, 0.07428386807441711, 0.04028827324509621, 0.029407696798443794, 0.054055847227573395, 0.05230886861681938, -0.031010109931230545, 0.005907146260142326, 0.0556052066385746, 0.011840774677693844, -0.014026502147316933, 0.009590055793523788, -0.07618232071399689, 0.07943598926067352, 0.015575905330479145, -0.013951631262898445, 0.11029195785522461, 0.0781882107257843, -0.04894213378429413, -1.4031407772563398e-05, -0.0052127111703157425, -0.014335749670863152, 0.001761345425620675, 0.05901962146162987, 0.0014778276672586799, 0.01818060874938965, 0.06327041238546371, 0.032311055809259415, -0.08511326462030411, 0.05002790689468384, -0.0033054249361157417, -0.014438709244132042, 0.02113744430243969, 0.018039241433143616, -0.01986449398100376, 0.0016447